In [ ]:
# Instalação dos requisitos
#!conda install -c conda-forge poppler
!pip install wget
!pip install bs4
!pip install requests
!pip install pyPdf2
!pip install pdftotext

In [ ]:
# Fonte de dados - Vitoria - ES

In [ ]:
# Lendo o site do diario oficial

from bs4 import BeautifulSoup
import requests

url = 'https://diariooficial.vitoria.es.gov.br'
r  = requests.get(url)

data = r.text
soup = BeautifulSoup(data)
lista_links = []
lista_datas = []

# Obtendo links dos pdfs através de web scrapping com o beautiful soup

for link in soup.find_all('a'):
    if str(link.get('href')).count('ExibirArquivo.aspx?') > 0: 
     #   print(link.get('href'))
        lista_links.append(str(url)+'/'+str(link.get('href')))

# Obtendo os nomes dos arquivos pdf, que no caso é no padrao Diario_Oficial_PMV_DD_MM_AAAA.pdf
# Pegamos apenas a data
    
for nomearq in soup.find_all('span'):
    if str(nomearq).count('publicado em') > 0: 
        aux = str(nomearq)
        aux = aux.replace('/','-')
     #   print(aux[len(aux)-17:len(aux)-7])
        lista_datas.append(aux[len(aux)-17:len(aux)-7])

print(lista_datas)

In [ ]:
import wget
import os


# Faz o download de cada pdf a partir do wget
# Se o arquivo já existe na pasta, incrementar o nome (pode haver mais de um diario por dia)

i = 0
for arquivo in lista_links:
  nomeArqSaida = 'c:\\temp\\vitoria_gov\\arq_' + lista_datas[i] + '.pdf'
  wget.download(arquivo)
  if not os.path.exists(nomeArqSaida):
    os.rename('2021.pdf',nomeArqSaida)
  else:
    st = str(i)
    os.rename('2021.pdf',nomeArqSaida+'('+st+')')
  i = i + 1;


In [ ]:
from PyPDF2 import PdfFileReader
import re
import os
import fitz 

#  Lendo o pdf e buscando textos relativos a nomeações


StringInicio = "EDITAL DE CONVOCAÇÃO E NOMEAÇÃO"
StringFim = "E CONVOCA os profissionais acima nomeados"

StringInicio2 = "Nomeia"

StringFim2 = "Art."

print("######")

#for data in lista_datas:
for file in os.listdir("C:\\tcc\\vitoria_gov"):
    print(file)
    if file.endswith(".pdf"):
        data =  str(os.path.basename(file))
        print("#######" + data)

        ##input = PdfFileReader("C:\\temp\\vitoria_gov\\arq_"+data+".pdf")
        input = PdfFileReader("C:\\tcc\\vitoria_gov\\"+data)
       # input = doc = fitz.open("C:\\tcc\\vitoria_gov\\"+data)
        NumPages = input.getNumPages()
        #NumPages = input.pageCount  
        print('arquivo ' + data)

        for i in range(0, NumPages):

            PageObj = input.getPage(i)
            Text = PageObj.extractText() 
            
         #   pg=input.loadPage(i) 
          #  Text=pg.getText() 
            
            ResSearch = re.search(StringInicio, Text) # busca o inicio do texto de nomeacao
            ResSearch2 = re.search(StringInicio2, Text)

            # Se houver ocorrencia nessa pagina de alguma das 2 strings, gravar em arquivo txt

            if ResSearch is not None:
                start = Text.find(StringInicio) + len(StringInicio) # posicao do caracter onde termina o texto inicial de nomeacao
                end = Text.find(StringFim)   # encontra o final do texto de nomeação
                lista = Text[start:end]     # obtem o texto entre sa marcacoes de inicio e fim

                if lista is not None:
                    arqtxt = "C:\\tcc\\vitoria_gov\\arq_"+data+".txt"
                    if not os.path.exists(arqtxt):
                        file = open(arqtxt, "w",encoding='utf-8') 
                        print('criando arquivo ' + arqtxt)
                    else:   # caso exista mais de uma ocorrencia por diario , cada um vira um txt diferente
                        file = open(arqtxt+".b", "w",encoding='utf-8')                 

                    file.write(lista) 
                    file.close() 


            if ResSearch2 is None:
                start2 = Text.find(StringInicio2) + len(StringInicio2)
                end2 = Text.find(StringFim2)
                lista2 = Text[start2:end2]

                if lista2 is None:
                    arqtxt = "C:\\tcc\\vitoria_gov\\arq_"+data+".txt"
                    if not os.path.exists(arqtxt):
                        file2 = open(arqtxt, "w",encoding='utf-8') 
                        print('criando arquivo ' + arqtxt)
                    else:   # caso exista mais de uma ocorrencia por diario , cada um vira um txt diferente
                        file = open(arqtxt+".b", "w",encoding='utf-8')                 
                    file2.write(lista2) 
                    file2.close()


In [ ]:
# Esta função retorna ocorrencias de um texto (no caso, localizar º que é a marca de posição no concurso)
# Adaptada de <<<inserir creditos>>>

def search_string_in_file(file_name, string_to_search):
    line_number = 0
    list_of_results = []
    with open(file_name, 'r',encoding="utf-8") as read_obj:
        for line in read_obj:
            # Ver se a linha está na string
            line_number += 1
            if string_to_search in line:
                # Se tem a substring º, verificar se à esquerda tem digitos (ou seja, classificacao em concurso)
                sti2 = line.rpartition(string_to_search)[0]
                sti2 = sti2[:2]
                for character in sti2:
                    if character.isdigit():
                        
                        # tratando os casos com duas ocorrencias na mesma linha (adicionar uma quebra de linha)
                        if line.count("º") >= 2:
                          #  print('tem uma linha com dois º')
                            line = line.replace("º","\n")
                            
                        # retirar as ocorrencias como  "Art. 1º"  , ou "Lei n º", que tem o caracter º mas nao desejamos
                        ex1="Art."
                        ex2="Lei nº"
                        ex3="Processo Nº"
                        if(ex1 not in line and ex2 not in line and ex3 not in line):
                            list_of_results.append(line) # adiciona à lista dos resultados
                        break
                        
    return list_of_results

In [ ]:
import os
import csv

# Criada uma lista total com todas as nomeacoes de todos diarios
# Cada elemento da lista total é uma lista com as nomeacoes do diario do dia

listaTotal = []
for file in os.listdir("C:\\temp\\vitoria_gov"):
    if file.endswith(".txt"):
        lista = search_string_in_file(os.path.join("C:\\temp\\vitoria_gov", file) , 'º')
 #       listaTotal.append(lista)
        
        with open('C:\\temp\\vitoria_gov\\'+file+'.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            for row in range(len(lista)):
                myList = []
                myList.append(lista[row])
                writer.writerow(myList)
            #    listaTotal.append(myList)
                listaTotal.append(lista[row])
        
    print(listaTotal)

In [ ]:
# Obtendo os cargos dos nomeados (apenas os nomes dos cargos)

import re
import os

# Exemplo de trecho:

#os candidatos habilitados no Concurso Público realizado pelo Edital
#n.º 001/2019, conforme abaixo identificados:
#PROFESSOR DE EDUCAÇÃO BÁSICA I - PEB I - 25 HORAS 
#Processo nº. 3329020/2020

StringCargoInicio = "conforme abaixo identificados:"
StringCargoFm = "HORAS"
Text = ""

listaCargos = []

for file in os.listdir("C:\\temp\\vitoria_gov"):
    if file.endswith(".txt"):  
        with open("C:\\temp\\vitoria_gov\\"+file, 'r',encoding="utf-8") as read_obj:
            print("arquivo " + file)
            Text = ""
            for line in read_obj:
                Text = Text + line

  # funcionava somente para um dos diarios - de janeiro              
  # expRegular = "conforme abaixo(.*?)cados:\n\n(.*?)HORAS"
  # cargo = re.findall(expRegular, Text,re.DOTALL) 
            
            expRegular = "(.*?)\s-\s[0-9]+\sHORAS\n"
            cargos = re.findall(expRegular, Text,re.MULTILINE) 
            print(cargos)
            listaCargos.append(cargos)


In [ ]:
# realiza o merge de pessoa com cargo e a data do concursom já que nao foi possivel associar com regex

# obtem o numero da linha onde o cargo aparece e o proximo cargo
# lista as pessoas que estap entre as linhas dos 2 cargos



cargo = ""

arq = 0
for file in os.listdir("C:\\temp\\vitoria_gov"):
    if file.endswith(".txt"):
        linha = 1
        linha2 = 1

        print(file)
        with open("C:\\temp\\vitoria_gov\\"+file, 'r',encoding="utf-8") as read_obj:
            
            #  loop para as linhas dos cargos nos arquivos

            cLinhas = []
            lCargos = []

            cargosNoArq = len(listaCargos[arq])
            print(str(cargosNoArq) + " cargos nesse arquivo")
            for cargo in listaCargos[arq]:
                read_obj.seek(0)  # Volta para o inicio do arquivo
                linha = 0
                for line in read_obj:
                    if(cargo in line):
                        print("cargo " + cargo + " na linha " + str(linha) + "::: " + line)
                        lCargos.append(cargo)
                        cLinhas.append(linha)
                    linha = linha + 1
            
            #  loop para as linhas que aparecem as pessoas nos arquivos
   
        with open("C:\\temp\\vitoria_gov\\"+file, 'r',encoding="utf-8") as read_obj:
            for pessoa in listaTotal:
                read_obj.seek(0)  # Volta para o inicio do arquivo
                linha2 = 0
                for line2 in read_obj:
                    if(pessoa in line2):
                        print(pessoa + " na linha " + str(linha2) + "::: " + line2)
                        for i, e in reversed(list(enumerate(cLinhas))):             
                            if(linha2 > int(e)):
                                row = [pessoa,lCargos[i],file]
                                print(" pessoa no cargo " + lCargos[i]+"\n")
                                print("##### " + str(linha2) + " " + str(cLinhas[i]) + " #####")
                                with open('C:\\temp\\vitoria_gov\\'+file+'_FINAL.csv', 'a', newline='') as csvfile:
                                    writer = csv.writer(csvfile, delimiter=';')
                                    writer.writerow(row)
                                break;
                    linha2 = linha2 + 1
            
        arq = arq + 1

In [ ]:
# Vitoria - obtem dos diarios mais recentes , cuja escrita mudou
# Mesma grafia do Serra2

# arq_19-08-2021.pdf
#D E C R E T A:
#Art. 1º. Nomeia Marcos Antônio da Rocha para exercer o
#cargo comissionado de Chefe de Equipe, PC-OP2, na Secretaria
#de Saúde, na forma do Art. 11, inciso III, da Lei n° 2.994/82.
#Art. 2°. Este Decreto entra em vigor na data de sua publicação.
#Palácio Jerônimo Monteiro, em 18 de agosto de 2021


In [ ]:
# API de leitura PDF

# nao funciona PyPDF2 - pdfFileReader

#input = PdfFileReader("C:\\Temp\\vitoria_gov\\arq_02-08-2021.pdf")  #NAO LÊ
#input = PdfFileReader("C:\\Temp\\vitoria_gov\\arq_07-07-2021.pdf")  #LÊ
#input = PdfFileReader("C:\\Temp\\vitoria_gov\\arq_30-06-2020.pdf")  #NAO LÊ
#input = PdfFileReader("C:\\Temp\\vitoria_gov\\arq_10-12-2020.pdf")  #LÊ

NumPages = input.getNumPages()

for i in range(0, NumPages):
    PageObj = input.getPage(i)
    Text = PageObj.extractText() 
    Text = Text.replace('\n','')
    print(Text)



#Este documento foi assinado digitalmente por MUNICIPIO DE VITORIA:27142058000126Para verificar a assinatura acesse o site http://diariooficial.vitoria.es.gov.br/ 
#e utilize a chave 08EBC452-78CA-4BFE-9767-788164B20093

In [ ]:
# fazer merge de Vitoria ES

import os

f = open('C:\\tcc\\vitoria_gov\\CONSOLIDADO.csv', 'w')
for file in os.listdir("C:\\tcc\\vitoria_gov"):
    ##     if os.path.isfile(os.path.join(root, item)):
    if file.endswith("FINAL.csv") and not file.startswith("CONSOLIDADO"):
        with open("C:\\tcc\\vitoria_gov\\"+file, 'r') as read_obj:
            for line in read_obj:
                line = line.replace("\n","")
                line = line.replace("\r","")
              #  line = line + ";Vitoria;ES"+"\n"
                line = line + "\n"
                f.write(line)
f.close()